In [1]:
import roboticstoolbox as rtb
import spatialmath as sm
import numpy as np
from swift import Swift


# Make and instance of the Swift simulator and open it
env = Swift()
env.launch(realtime=True)

# Make a panda model and set its joint angles to the ready joint configuration
panda = rtb.models.Panda()
panda.q = panda.qr

# Set a desired and effector pose an an offset from the current end-effector pose
Tep = panda.fkine(panda.q) * sm.SE3.Tx(0.2) * sm.SE3.Ty(0.2) * sm.SE3.Tz(0.45)

# Add the robot to the simulator
env.add(panda)

# Simulate the robot while it has not arrived at the goal
arrived = False
while not arrived:

    # Work out the required end-effector velocity to go towards the goal
    v, arrived = rtb.p_servo(panda.fkine(panda.q), Tep, 1)
    
    # Set the Panda's joint velocities
    panda.qd = np.linalg.pinv(panda.jacobe(panda.q)) @ v
    
    # Step the simulator by 50 milliseconds
    env.step(0.05)

In [67]:
from roboticstoolbox import *
import numpy as np

class GEN3_LITE(ERobot):

    def __init__(self):

        # links, name, urdf_string, urdf_filepath = self.URDF_read("/home/riot/kinova_gen3_lite/src/ggcnn/data/xacro/robots/GEN3-LITE.urdf")
        links, name, urdf_string, urdf_filepath = self.URDF_read("/home/riot/kinova_gen3_lite/src/ros_kortex/kortex_description/robots/gen3_lite_gen3_lite_2f.xacro")
    
        super().__init__(
            links,
            name='gen3-lite',
            urdf_string=urdf_string,
            urdf_filepath=urdf_filepath,
            manufacturer="Kinova",
            gripper_links=links[7]
        )

        self.addconfiguration(
            "qr", np.array([0, 0, 0, 0, 0, 0])
        )



In [68]:
robot = GEN3_LITE()
print(robot)

ERobot: gen3-lite (by Kinova), 6 joints (RRRRRR), 1 gripper, dynamics, geometry, collision
┌─────┬────────────────────┬───────┬──────────────────┬───────────────────────────────────────────────────┐
│link │        link        │ joint │      parent      │                ETS: parent to link                │
├─────┼────────────────────┼───────┼──────────────────┼───────────────────────────────────────────────────┤
│   0 │ base_link          │       │ BASE             │ SE3()                                             │
│   1 │ shoulder_link      │     0 │ base_link        │ SE3(0, 0, 0.1283) ⊕ Rz(q0)                        │
│   2 │ arm_link           │     1 │ shoulder_link    │ SE3(0, -0.03, 0.115; 90°, -0°, 0°) ⊕ Rz(q1)       │
│   3 │ forearm_link       │     2 │ arm_link         │ SE3(0, 0.28, 0; 180°, -0°, 0°) ⊕ Rz(q2)           │
│   4 │ lower_wrist_link   │     3 │ forearm_link     │ SE3(0, -0.14, 0.02; 90°, -0°, 0°) ⊕ Rz(q3)        │
│   5 │ upper_wrist_link   │     4 │ lower_wr

In [69]:
robot.grippers

[Gripper(['Link([ET.SE3(T=array([[1., 0., 0., 0.],\n       [0., 1., 0., 0.],\n       [0., 0., 1., 0.],\n       [0., 0., 0., 1.]]))], name = "dummy_link", parent="end_effector_link")', 'Link([ET.SE3(T=array([[ 4.89658886e-12, -1.00000000e+00,  0.00000000e+00,\n         0.00000000e+00],\n       [ 1.00000000e+00,  4.89658886e-12,  0.00000000e+00,\n         0.00000000e+00],\n       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,\n         1.30000000e-01],\n       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,\n         1.00000000e+00]]))], name = "tool_frame", parent="dummy_link")', 'Link([ET.SE3(T=array([[1., 0., 0., 0.],\n       [0., 1., 0., 0.],\n       [0., 0., 1., 0.],\n       [0., 0., 0., 1.]]))], name = "gripper_base_link", parent="dummy_link", m=0.14, r=[4.03e-06, 1.08e-05, 0.0414], I=[0.000116, 0.000103, 9.71e-05, 1e-08, 4e-08, 0], Jm=0, B=0, Tc=[0, 0], G=0)', 'Link([ET.SE3(T=array([[-3.6732051e-06,  0.0000000e+00,  1.0000000e+00,  0.0000000e+00],\n       [ 0.0000000e+00,

In [70]:
robot.dynamics()

┌──────────────────┬────────┬──────────────────────────────┬────────────────────────────────────────────────────────────────┬────┬────┬────────┬────┐
│        j         │   m    │              r               │                               I                                │ Jm │ B  │   Tc   │ G  │
├──────────────────┼────────┼──────────────────────────────┼────────────────────────────────────────────────────────────────┼────┼────┼────────┼────┤
│        base_link │  1.15  │  0.00244,  0.000156,  0.0862 │  0.00336,  0.00331,  0.000772,  3.9e-07,  1.91e-06,  0.00011   │  0 │  0 │  0,  0 │  0 │
│    shoulder_link │  0.96  │  2.48e-05,  0.0221,  0.0994  │  0.00166,  0.0014,  0.000895,  2e-08,  0.000349,  3.6e-07      │  0 │  0 │  0,  0 │  0 │
│         arm_link │  1.18  │  0.03,  0.212,  0.0453       │  0.0115,  0.00103,  0.0113,  1e-06,  0.00141,  1.6e-07         │  0 │  0 │  0,  0 │  0 │
│     forearm_link │  0.598 │  0.0302,  0.095,  0.00736    │  0.00163,  0.000298,  0.00169,  7.11e-0

In [71]:
robot.plot(np.array([0, 0, 0, 0, 0, 0]))

Swift backend, t = 0.05, scene:
  gen3-lite